## Visión por Computadora 1 - Cohorte 17
## Trabajo Práctico 2
## Paola Cartala - Florentino Arias

## Parte 1: Implementar un algoritmo de enfoque que calcule la métrica descrita en el paper 'Image Sharpness Measure for Blurred Images in Frequency Domain'.
Implementamos la métrica de enfoque para un frame usando la Transformada Rápida de Fourier (FFT).

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Leer el video
video_path = 'focus_video.mov'
cap = cv2.VideoCapture(video_path)

focus_scores = []
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertir el frame a escala de grises
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calcular la FFT
    f = np.fft.fft2(gray_frame)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    
    # Calcular la métrica: suma de las frecuencias altas (evitar el centro de la FFT)
    center_x, center_y = magnitude_spectrum.shape[1] // 2, magnitude_spectrum.shape[0] // 2
    roi_radius = 20  # Tamaño del centro a evitar (puedes ajustar según el tamaño de la imagen)
    mask = np.ones_like(magnitude_spectrum)
    cv2.circle(mask, (center_x, center_y), roi_radius, 0, -1)
    
    high_freq_values = magnitude_spectrum * mask
    focus_measure = np.sum(high_freq_values)
    focus_scores.append(focus_measure)

    frame_count += 1

cap.release()

# Graficar la evolución de la métrica frame a frame
plt.plot(focus_scores)
plt.title('Evolución de la métrica de enfoque frame a frame')
plt.xlabel('Frame')
plt.ylabel('Métrica de Enfoque')
plt.show()


### Parte 1a: Medición sobre todo el Frame
El código de arriba ya calcula la métrica sobre todo el frame, y la gráfica muestra cómo varía la nitidez a lo largo del video.

### Parte 1b: Medición sobre una ROI en el Centro del Frame
Calculamos la métrica solo en una región de interés (ROI) en el centro del frame. El área de la ROI será del 5% o 10% del área total del frame.

In [ ]:
cap = cv2.VideoCapture(video_path)
focus_scores_roi = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertir el frame a escala de grises
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Definir ROI en el centro del frame (10% del área total)
    h, w = gray_frame.shape
    roi_h, roi_w = int(0.1 * h), int(0.1 * w)
    start_x, start_y = (w - roi_w) // 2, (h - roi_h) // 2
    roi = gray_frame[start_y:start_y+roi_h, start_x:start_x+roi_w]
    
    # Calcular la FFT y la métrica para la ROI
    f = np.fft.fft2(roi)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    
    # Calcular la métrica como antes
    focus_measure = np.sum(magnitude_spectrum)
    focus_scores_roi.append(focus_measure)

cap.release()

# Graficar la evolución de la métrica en la ROI
plt.plot(focus_scores_roi)
plt.title('Evolución de la métrica de enfoque sobre ROI en el centro')
plt.xlabel('Frame')
plt.ylabel('Métrica de Enfoque ROI')
plt.show()


### Parte 1c: Medición sobre una Matriz de Enfoque (NxM)
Para esta parte vamos a dividir cada frame en una matriz de elementos rectangulares y a calcular la métrica para cada sección.

In [ ]:
N_values = [3, 5, 7]  # Probar varios valores para N y M
M_values = [3, 5]

cap = cv2.VideoCapture(video_path)

for N, M in zip(N_values, M_values):
    focus_scores_grid = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convertir el frame a escala de grises
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        h, w = gray_frame.shape
        step_h, step_w = h // N, w // M
        
        total_focus = 0
        # Iterar sobre la matriz NxM
        for i in range(N):
            for j in range(M):
                roi = gray_frame[i*step_h:(i+1)*step_h, j*step_w:(j+1)*step_w]
                
                # Calcular la FFT y la métrica para la ROI
                f = np.fft.fft2(roi)
                fshift = np.fft.fftshift(f)
                magnitude_spectrum = np.abs(fshift)
                
                # Sumar las métricas para cada sección
                total_focus += np.sum(magnitude_spectrum)

        focus_scores_grid.append(total_focus)

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reiniciar video para la siguiente iteración

    # Graficar la evolución de la métrica sobre la matriz
    plt.plot(focus_scores_grid)
    plt.title(f'Evolución de la métrica de enfoque para la matriz {N}x{M}')
    plt.xlabel('Frame')
    plt.ylabel('Métrica de Enfoque Matriz')
    plt.show()

cap.release()


Visualización de la ROI o Matriz Superpuesta al Video
Para marcar los frames en rojo o verde según el enfoque:

In [ ]:
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter('focus_output.avi', cv2.VideoWriter_fourcc(*'XVID'), 30, (int(cap.get(3)), int(cap.get(4))))

for i, score in enumerate(focus_scores_roi):  # Usando la métrica ROI como ejemplo
    ret, frame = cap.read()
    if not ret:
        break
    
    color = (0, 255, 0) if score == max(focus_scores_roi) else (0, 0, 255)
    # Dibujar el rectángulo de la ROI
    cv2.rectangle(frame, (start_x, start_y), (start_x+roi_w, start_y+roi_h), color, 2)
    
    out.write(frame)

cap.release()
out.release()


## Parte 2: Cambiar la Métrica de Enfoque
En esta parte podemos utilizar una métrica diferente, como la de la varianza de Laplaciano:

In [ ]:
cap = cv2.VideoCapture(video_path)
laplacian_focus_scores = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
    laplacian_focus_scores.append(laplacian_var)

cap.release()

# Graficar la evolución de la nueva métrica
plt.plot(laplacian_focus_scores)
plt.title('Evolución de la métrica de enfoque con Laplaciano')
plt.xlabel('Frame')
plt.ylabel('Varianza del Laplaciano')
plt.show()
